In [1]:
import time
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup

print(selenium.__version__)
# prints 4.11.2


4.11.2


In [2]:
service = webdriver.ChromeService('chromedriver.exe')
driver = webdriver.Chrome(service=service)

# replace url with something else
url = "https://www.google.com/maps/place/Basilica+Minore+del+Santo+Ni%C3%B1o+de+Cebu+Pilgrim+Center/@10.2940228,123.8999268,17z/data=!4m6!3m5!1s0x33a999575a49c9fb:0xe2caa9445367b234!8m2!3d10.2940225!4d123.9025019!16s%2Fg%2F11bzvzxdvd?entry=ttu"

driver.get(url)

In [ ]:
location_name = driver.find_element("xpath", '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[1]/h1').text
location_name = location_name.replace(" ", "_")
print(location_name)
time.sleep(1)

In [40]:
driver.find_element("xpath", '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]').click()
time.sleep(3)
driver.find_element("xpath", '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button').click()
time.sleep(1)
driver.find_element('xpath', '//*[@id="action-menu"]/div[2]').click()

KeyboardInterrupt: 

: 

In [39]:
total_number_of_reviews = driver.find_element('xpath', '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[2]').text.split(" ")[0]
total_number_of_reviews = int(total_number_of_reviews.replace(',', '')) if ',' in total_number_of_reviews else int(total_number_of_reviews)

scrollable_div = driver.find_element('xpath', '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')

length = 0
for i in range(0, (round(total_number_of_reviews/10 - 1))):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div )
    if length == 180: 
        break
    length += 1
    # print(length)
    time.sleep(1)

TimeoutException: Message: script timeout
  (Session info: chrome=116.0.5845.111)
Stacktrace:
	GetHandleVerifier [0x00007FF7125652A2+57122]
	(No symbol) [0x00007FF7124DEA92]
	(No symbol) [0x00007FF7123AE25D]
	(No symbol) [0x00007FF71241F314]
	(No symbol) [0x00007FF712406FDA]
	(No symbol) [0x00007FF71241EB82]
	(No symbol) [0x00007FF712406DB3]
	(No symbol) [0x00007FF7123DD2B1]
	(No symbol) [0x00007FF7123DE494]
	GetHandleVerifier [0x00007FF71280EF82+2849794]
	GetHandleVerifier [0x00007FF712861D24+3189156]
	GetHandleVerifier [0x00007FF71285ACAF+3160367]
	GetHandleVerifier [0x00007FF7125F6D06+653702]
	(No symbol) [0x00007FF7124EA208]
	(No symbol) [0x00007FF7124E62C4]
	(No symbol) [0x00007FF7124E63F6]
	(No symbol) [0x00007FF7124D67A3]
	BaseThreadInitThunk [0x00007FFA71B07614+20]
	RtlUserThreadStart [0x00007FFA73AA26B1+33]


In [31]:
def get_review_summary(result_set, user_name, user_rating, user_timestamp, user_text):
    review_data = []

    for result in result_set:
        review_dict = {}
        try:
            review_dict['review_user'] = result.find('div', class_=user_name).text
        except AttributeError:
            review_dict['review_user'] = "N/A"

        try:
            review_dict['review_rate'] = result.find('span', class_=user_rating)["aria-label"]
        except (AttributeError, KeyError):
            review_dict['review_rate'] = "N/A"

        try:
            review_dict['review_time'] = result.find('span', class_=user_timestamp).text
        except AttributeError:
            review_dict['review_time'] = "N/A"

        try:
            review_dict['review_text'] = result.find('span', class_=user_text).text
        except AttributeError:
            review_dict['review_text'] = "N/A"

        review_data.append(review_dict)

    return review_data

In [36]:
import csv

# alter these variables as see fit
review_container = 'jJc9Ad'
user_name = 'd4r55'
user_rating = 'kvMYJc'
user_timestamp = 'rsqaWe'
user_text = 'wiI7pd'

response = BeautifulSoup(driver.page_source, 'html.parser')
reviews = response.find_all('div', class_=review_container)
review_data = get_review_summary(reviews, user_name, user_rating, user_timestamp, user_text)

csv_file_path = f"csv/{location_name}.csv"

with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['review_user', 'review_rate', 'review_time', 'review_text']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()  # Write the header row
    
    for review in review_data:
        writer.writerow(review)


# identical classname: jJc9Ad 
# name: d4r55  
# timestamp: rsqaWe
# rating: kvMYJc ["aria-label"]